# Trabalho final

Simone Gomes

1931133125

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
# Carregando os dados
df = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

df.shape, test.shape

9557 linhas pra treinar e 23856 linhas pra prever

In [ ]:
# Juntando os dataframes
df_all = df.append(test)

df_all.shape

In [ ]:
df_all.head().T


In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

coluna com tipo texto que devem ser tratadas.
id e idhobar não precisam ser tratadas

In [ ]:
# Analisando os dados da coluna edjefa
df_all['edjefa'].value_counts()

In [ ]:
# Analisando os dados da coluna edjefe
df_all['edjefe'].value_counts()

verificou-se que 12818 chefes de família não tem escolaridade. Será feito um tratamento no campo NO e yes

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
# nas colunas edjefa e edjefe
mapeamento = {'yes': 1, 'no': 0}

df_all['edjefa'] = df_all['edjefa'].replace(mapeamento).astype(int)
df_all['edjefe'] = df_all['edjefe'].replace(mapeamento).astype(int)

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Olhando a coluna dependency
df_all['dependency'].value_counts()

sera feito um tratamento nos dados no e yes

## Tratamento nos dados

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
# na coluna dependency
df_all['dependency'] = df_all['dependency'].replace(mapeamento).astype(float)

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum().sort_values()

#### será feito tratamento nos dados 'meaneduc', 'v2a1','v18q1' , 'rez_esc' e 'SQBmeaned'

In [ ]:
#df_all['meaneduc'].fillna(df_all['meaneduc'].mean(), inplace=True) 
#df_all['v2a1'].fillna(df_all['v2a1'].mean(), inplace=True) 
#df_all['rez_esc'].fillna(df_all['rez_esc'].mean(), inplace=True) 
#df_all['v18q1'].fillna(0, inplace=True)
#df_all['SQBmeaned'].fillna(df_all['SQBmeaned'].mean(), inplace=True)

In [ ]:
df_all['meaneduc'].fillna(df_all['meaneduc'].median(), inplace=True) 
df_all['v2a1'].fillna(df_all['v2a1'].median(), inplace=True) 
df_all['rez_esc'].fillna(df_all['rez_esc'].median(), inplace=True) 
df_all['v18q1'].fillna(0, inplace=True)
df_all['SQBmeaned'].fillna(df_all['SQBmeaned'].median(), inplace=True)

In [ ]:
df_all.isnull().sum().sort_values()

In [ ]:
# Prrenchendo com -1 todos os valores nulos
#df_all['v2a1'].fillna(-1, inplace=True)
#df_all['meaneduc'].fillna(-1, inplace=True)
#df_all['rez_esc'].fillna(-1, inplace=True)
#df_all['SQBmeaned'].fillna(-1, inplace=True)

In [ ]:
# Prrenchendo com 0 todos os valores nulos
#df_all['v18q1'].fillna(0, inplace=True)

In [ ]:
# Separando as colunas para treinamento
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target','edjefe', 'edjefa']]
#'edjefe', 'edjefa'

In [ ]:
# Separar os dataframes
train, test = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]

train.shape, test.shape

# Treinando os modelos

### Random Forest

In [ ]:
# Instanciando o random forest classifier
#from sklearn.ensemble import RandomForestClassifier

#rf = RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=42, n_estimators=200)
#0.36736




In [ ]:
rf.fit(train[feats], train['Target'])

In [ ]:
accuracy_score(train['Target'], rf.predict(train[feats]))


In [ ]:
preds_val = rf.predict(train[feats])

In [ ]:
# importando a bilbioteca para plotar o gráfico de Matriz de Confusão
#from sklearn.metrics import confusion_matrix
#confusion_matrix(train, preds_val)


# importando a bilbioteca para plotar o gráfico de Matriz de Confusão
 import scikitplot as skplt



#### Foi realizado o teste e o melhor score conseguido foi de 0.36736. Foi feito testes inserindo média, mediana e moda nos dados missing.

# XGBboost

In [ ]:
#from xgboost import XGBClassifier
rf2 = XGBClassifier(n_estimators=200, learning_rate=0.09, random_state=42)
# 0.38378


In [ ]:
rf2.fit(train[feats], train['Target'])
accuracy_score(train['Target'], rf2.predict(train[feats]))


#### Foi realizado o teste e o melhor score conseguido foi de 0.38378. Foi feito testes inserindo média, mediana e moda nos dados missing.

# GBM

In [ ]:
#from sklearn.ensemble import GradientBoostingClassifier
rf3 = GradientBoostingClassifier(n_estimators=200, learning_rate=1.0, max_depth=1, random_state=42)
#0.37065

In [ ]:
rf3.fit(train[feats], train['Target'])
accuracy_score(train['Target'], rf3.predict(train[feats]))

Foi realizado o teste e o melhor score conseguido foi de 0.37065. Foi feito testes inserindo média, mediana e moda nos dados missing.

# Random Foret Classifier 

In [ ]:
# Copiando do campeão

rf = RandomForestClassifier(max_depth=None, random_state=42, n_jobs=4, n_estimators=700,
                            min_impurity_decrease=1e-3, min_samples_leaf=2,
                            verbose=0, class_weight='balanced')

In [ ]:
rf.fit(train[feats], train['Target'])
accuracy_score(train['Target'], rf.predict(train[feats]))

Foi realizado o teste e o melhor score conseguido foi de **0.43480**. Foi feito testes inserindo média, mediana e moda nos dados missing.

In [ ]:
# Treinando o modelo
rf.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = rf.predict(test[feats]).astype(int)

In [ ]:
# Vamos verificar as previsões
test['Target'].value_counts(normalize=True)

In [ ]:
# Criando o arquivo para submissão
test[['Id', 'Target']].to_csv('submission.csv', index=False)

# Relatório

### Foram realizados 4 testes com modelos diferentes e o melhor resultado foi com o random forest balanceado. 
https://www.kaggle.com/simonesymon/1931133125/log?scriptVersionId=42034868
### Realizei vários testes alterando as variaveis para usar a média, mediana e moda. Alterei também a questão da retirada de variaveis para gerar os modelos de treino e teste. Foram praticamente 2 dias testando.
### Achei super interessante fazer esse trabalho e cheguei a conclusão que preciso estudar mais para ter domínio sobre os modelos a serem construídos. Tive dificuldades por conta do idioma, meu inglês está enferrujado e vou ter que correr atrás.



